In [ ]:
import rdflib
import pandas as pd
from rdflib import Graph, URIRef, Literal, Namespace

In [ ]:
EX = rdflib.Namespace("http://example.org/medsur.rdf#")
RDF = rdflib.Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
RDFS = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#")

In [ ]:
g = Graph()
g.parse("medsur.ttl", format="turtle")

In [ ]:
def Query_1(g):
    """Find all unique drugs."""
    res = g.query("""
    SELECT DISTINCT ?drug where { 
    ?drug rdf:type ex:Drug . 
    }
    """, initNs={"ex": EX, "rdf": RDF})
    return list(res)


unique_drugs = Query_1(g)
print(len(unique_drugs))

In [ ]:
def Query_2a(g):
    """Find all unique PTCodes."""
    res = g.query("""
    SELECT DISTINCT ?ptcode where {
    ?ptcode rdf:type ex:PT .
    } 
    """, initNs={"ex": EX, "rdf": RDF})
    return list(res)

unique_PTC = Query_2a(g)
print(len(unique_PTC))

In [ ]:
def Query_2b(g):
    """Find all unique SOCCodes."""
    res = g.query("""
    SELECT DISTINCT ?soccode where {
    ?soccode rdf:type ex:SOC .
    } 
    """, initNs={"ex": EX, "rdf": RDF})
    return list(res)

unique_PTC = Query_2b(g)
print(len(unique_PTC))

In [ ]:
def Query3(g):
    """Count most frequent outcome for patient weight group."""
    res = g.query("""
    SELECT ?weightgroup ?outcome (COUNT(?outcome) AS ?count) WHERE {
    ?patient wtm:hasWeightGroup ?weightgroup .
    ?patient wtm:hasOutcome ?outcome .
    } GROUP BY ?outcome ?weightgroup ORDER BY DESC(?count)
    """, initNs={"wtm": EX})
    return list(res)

list_of_outcome_groups = Query3(g)
df_outcome_groups = pd.DataFrame(list_of_outcome_groups)
df_outcome_groups.columns = ['weight', 'outcome_group', 'frequency']
df_outcome_groups = df_outcome_groups.sort_values(by=['weight', 'frequency'], ascending=False)

# create new dataframe with top 3 outcome_groups for each weight with columns weight, outcome_group1, count1, outcome_group2, count2, outcome_group3, count3
df_outcome_groups_top3 = pd.DataFrame(columns=['weight', 'outcome_group1', 'frequency1', 'outcome_group2', 'frequency2', 'outcome_group3', 'frequency3'])

for weight in df_outcome_groups['weight'].unique():
    df_temp = df_outcome_groups[df_outcome_groups['weight'] == weight]
    df_temp = df_temp.head(3)
        
    # rescale count values to be between 0 and 1	
    df_temp['frequency'] = df_temp['frequency'].astype(float)
    df_temp['frequency'] = df_temp['frequency'] / df_temp['frequency'].sum()
    
    # add new row to df_outcome_groups_top3 if there are at least 3 outcome_groups for the weight
    if len(df_temp) == 3:
        df_outcome_groups_top3 = df_outcome_groups_top3.append({'weight': weight, 'outcome_group1': df_temp.iloc[0]['outcome_group'], 'frequency1': df_temp.iloc[0]['frequency'], 'outcome_group2': df_temp.iloc[1]['outcome_group'], 'frequency2': df_temp.iloc[1]['frequency'], 'outcome_group3': df_temp.iloc[2]['outcome_group'], 'frequency3': df_temp.iloc[2]['frequency']}, ignore_index=True)


# only select part after last slash
df_outcome_groups_top3['weight'] = df_outcome_groups_top3['weight'].apply(lambda x: x.split('/')[-1])
df_outcome_groups_top3['outcome_group1'] = df_outcome_groups_top3['outcome_group1'].apply(lambda x: x.split('/')[-1])
df_outcome_groups_top3['outcome_group2'] = df_outcome_groups_top3['outcome_group2'].apply(lambda x: x.split('/')[-1])
df_outcome_groups_top3['outcome_group3'] = df_outcome_groups_top3['outcome_group3'].apply(lambda x: x.split('/')[-1])

# remove row with weight 'nan'
df_outcome_groups_top3 = df_outcome_groups_top3[df_outcome_groups_top3['weight'] != 'nan']

print(df_outcome_groups_top3.head())

# save as csv
if not os.path.exists('output'):
    os.makedirs('output')
df_outcome_groups_top3.to_csv('output/df_outcome_weight_top3.csv', index=False)

In [ ]:
def Query4(g):
    """Count most frequent outcome for patient age group."""
    res = g.query("""
    SELECT ?agegroup ?outcome (COUNT(?outcome) AS ?count) WHERE {
    ?patient wtm:hasAgeGroup ?agegroup .
    ?patient wtm:hasOutcome ?outcome .
    } GROUP BY ?outcome ?agegroup ORDER BY DESC(?count)
    """, initNs={"wtm": EX})
    return list(res)

list_of_outcome_groups = Query4(g)
df_outcome_groups = pd.DataFrame(list_of_outcome_groups)
df_outcome_groups.columns = ['age', 'outcome_group', 'frequency']
df_outcome_groups = df_outcome_groups.sort_values(by=['age', 'frequency'], ascending=False)

# create new dataframe with top 3 outcome_groups for each age with columns age, outcome_group1, count1, outcome_group2, count2, outcome_group3, count3
df_outcome_groups_top3 = pd.DataFrame(columns=['age', 'outcome_group1', 'frequency1', 'outcome_group2', 'frequency2', 'outcome_group3', 'frequency3'])

for age in df_outcome_groups['age'].unique():
    df_temp = df_outcome_groups[df_outcome_groups['age'] == age]
    df_temp = df_temp.head(3)
        
    # rescale count values to be between 0 and 1	
    df_temp['frequency'] = df_temp['frequency'].astype(float)
    df_temp['frequency'] = df_temp['frequency'] / df_temp['frequency'].sum()
    
    # add new row to df_outcome_groups_top3 if there are at least 3 outcome_groups for the age
    if len(df_temp) == 3:
        df_outcome_groups_top3 = df_outcome_groups_top3.append({'age': age, 'outcome_group1': df_temp.iloc[0]['outcome_group'], 'frequency1': df_temp.iloc[0]['frequency'], 'outcome_group2': df_temp.iloc[1]['outcome_group'], 'frequency2': df_temp.iloc[1]['frequency'], 'outcome_group3': df_temp.iloc[2]['outcome_group'], 'frequency3': df_temp.iloc[2]['frequency']}, ignore_index=True)


# only select part after last slash
df_outcome_groups_top3['age'] = df_outcome_groups_top3['age'].apply(lambda x: x.split('/')[-1])
df_outcome_groups_top3['outcome_group1'] = df_outcome_groups_top3['outcome_group1'].apply(lambda x: x.split('/')[-1])
df_outcome_groups_top3['outcome_group2'] = df_outcome_groups_top3['outcome_group2'].apply(lambda x: x.split('/')[-1])
df_outcome_groups_top3['outcome_group3'] = df_outcome_groups_top3['outcome_group3'].apply(lambda x: x.split('/')[-1])

# replace age with boomers, genx, millenials, genz
df_outcome_groups_top3['age'] = df_outcome_groups_top3['age'].replace('65_above ', 'Boomers')
df_outcome_groups_top3['age'] = df_outcome_groups_top3['age'].replace('45_64', 'GenX')
df_outcome_groups_top3['age'] = df_outcome_groups_top3['age'].replace('25_44', 'Millenials')
df_outcome_groups_top3['age'] = df_outcome_groups_top3['age'].replace('18_24', 'GenZ')

# remove row with age 'nan'
df_outcome_groups_top3 = df_outcome_groups_top3[df_outcome_groups_top3['age'] != 'nan']

print(df_outcome_groups_top3.head())

# save as csv
if not os.path.exists('output'):
    os.makedirs('output')
df_outcome_groups_top3.to_csv('output/df_outcome_age_top3.csv', index=False)

In [ ]:
def Query5(g):
    """List most common symptom_groups for all unique drugs. Return the top 10 most common symptom_groups for each drug."""
    res = g.query("""
    select ?drug ?symptom_group (count(?symptom_group) as ?count) where {
    ?patient wtm:isGivenDrug ?drug .
    ?patient wtm:hasSOC ?symptom_group . 
    }
    group by ?drug ?symptom_group
    order by desc(?count)   
    """, initNs={"wtm": EX})

    return list(res)

list_of_symptom_groups = Query5(g)
df_symptom_groups = pd.DataFrame(list_of_symptom_groups)
df_symptom_groups.columns = ['drug', 'symptom_group', 'frequency']
df_symptom_groups = df_symptom_groups.sort_values(by=['drug', 'frequency'], ascending=False)

# create new dataframe with top 3 symptom_groups for each drug with columns drug, symptom_group1, count1, symptom_group2, count2, symptom_group3, count3
df_symptom_groups_top3 = pd.DataFrame(columns=['drug', 'symptom_group1', 'frequency1', 'symptom_group2', 'frequency2', 'symptom_group3', 'frequency3'])

for drug in df_symptom_groups['drug'].unique():
    df_temp = df_symptom_groups[df_symptom_groups['drug'] == drug]
    df_temp = df_temp.head(3)
        
    # rescale count values to be between 0 and 1	
    df_temp['frequency'] = df_temp['frequency'].astype(float)
    df_temp['frequency'] = df_temp['frequency'] / df_temp['frequency'].sum()
    
    # add new row to df_symptom_groups_top3 if there are at least 3 symptom_groups for the drug
    if len(df_temp) == 3:
        df_symptom_groups_top3 = df_symptom_groups_top3.append({'drug': drug, 'symptom_group1': df_temp.iloc[0]['symptom_group'], 'frequency1': df_temp.iloc[0]['frequency'], 'symptom_group2': df_temp.iloc[1]['symptom_group'], 'frequency2': df_temp.iloc[1]['frequency'], 'symptom_group3': df_temp.iloc[2]['symptom_group'], 'frequency3': df_temp.iloc[2]['frequency']}, ignore_index=True)


# remove url 
df_symptom_groups_top3['drug'] = df_symptom_groups_top3['drug'].str.replace('http://www.medsur.org/drug/', '')
df_symptom_groups_top3['symptom_group1'] = df_symptom_groups_top3['symptom_group1'].str.replace('http://www.medsur.org/soc/', '')
df_symptom_groups_top3['symptom_group2'] = df_symptom_groups_top3['symptom_group2'].str.replace('http://www.medsur.org/soc/', '')
df_symptom_groups_top3['symptom_group3'] = df_symptom_groups_top3['symptom_group3'].str.replace('http://www.medsur.org/soc/', '')

# replace drug codes with drug names from drug_dict
df_symptom_groups_top3['drug'] = df_symptom_groups_top3['drug'].map(drug_dict)

# replace symptom_group codes with symptom_group names from symptom_group_dict
df_symptom_groups_top3['symptom_group1'] = df_symptom_groups_top3['symptom_group1'].map(soc_dict)
df_symptom_groups_top3['symptom_group2'] = df_symptom_groups_top3['symptom_group2'].map(soc_dict)
df_symptom_groups_top3['symptom_group3'] = df_symptom_groups_top3['symptom_group3'].map(soc_dict)

# save as csv
if not os.path.exists('output'):
    os.makedirs('output')
df_symptom_groups_top3.to_csv('output/df_symptom_groups_top3.csv', index=False)

In [ ]:
def Query6(g):
    """List most common symptom for all unique drugs. Return the top 10 most common symptom for each drug."""
    res = g.query("""
    select ?drug ?symptom (count(?symptom) as ?count) where {
    ?patient wtm:isGivenDrug ?drug .
    ?patient wtm:hasPT ?symptom . 
    }
    group by ?drug ?symptom
    order by desc(?count)   
    """, initNs={"wtm": EX})

    return list(res)

list_of_symptom = Query6(g)
df_symptom = pd.DataFrame(list_of_symptom)
df_symptom.columns = ['drug', 'symptom', 'frequency']
df_symptom = df_symptom.sort_values(by=['drug', 'frequency'], ascending=False)

# create new dataframe with top 3 symptom for each drug 
df_symptom_top3 = pd.DataFrame(columns=['drug', 'symptom1', 'frequency1', 'symptom2', 'frequency2', 'symptom3', 'frequency3'])

for drug in df_symptom['drug'].unique():
    df_temp = df_symptom[df_symptom['drug'] == drug]
    df_temp = df_temp.head(3)
        
    # rescale count values to be between 0 and 1	
    df_temp['frequency'] = df_temp['frequency'].astype(float)
    df_temp['frequency'] = df_temp['frequency'] / df_temp['frequency'].sum()
    
    if len(df_temp) == 3:
        df_symptom_top3 = df_symptom_top3.append({'drug': drug, 'symptom1': df_temp.iloc[0]['symptom'], 'frequency1': df_temp.iloc[0]['frequency'], 'symptom2': df_temp.iloc[1]['symptom'], 'frequency2': df_temp.iloc[1]['frequency'], 'symptom3': df_temp.iloc[2]['symptom'], 'frequency3': df_temp.iloc[2]['frequency']}, ignore_index=True)

# remove url
df_symptom_top3['drug'] = df_symptom_top3['drug'].str.replace('http://www.medsur.org/drug/', '')
df_symptom_top3['symptom1'] = df_symptom_top3['symptom1'].str.replace('http://www.medsur.org/symptom/', '')
df_symptom_top3['symptom2'] = df_symptom_top3['symptom2'].str.replace('http://www.medsur.org/symptom/', '')
df_symptom_top3['symptom3'] = df_symptom_top3['symptom3'].str.replace('http://www.medsur.org/symptom/', '')

# replace drug codes with drug names from drug_dict
df_symptom_top3['drug'] = df_symptom_top3['drug'].map(drug_dict)

# replace symptom codes with symptom names from symptom_dict
df_symptom_top3['symptom1'] = df_symptom_top3['symptom1'].map(pt_dict)
df_symptom_top3['symptom2'] = df_symptom_top3['symptom2'].map(pt_dict)
df_symptom_top3['symptom3'] = df_symptom_top3['symptom3'].map(pt_dict)

# save as csv
if not os.path.exists('output'):
    os.makedirs('output')
df_symptom_top3.to_csv('output/df_symptom_top3.csv', index=False)